<a href="https://colab.research.google.com/github/christiane-villaroel/credit-card-fraud-detection/blob/main/clean_data_2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

Loading Data

In [ ]:
try:
  df=pd.read_csv('https://media.githubusercontent.com/media/christiane-villaroel/credit-card-fraud-detection/main/transactions.csv')
  print("File read successfully!\nYay!")
except pd.errors.ParserError as e:
  print(f'error reading file: {e}')
  with open('https://media.githubusercontent.com/media/christiane-villaroel/credit-card-fraud-detection/main/transactions.csv','r') as f:
    for line in f:
      print(line, end ='')

File read successfully!
Yay!


Inpecting My Data

In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


isFlaggedfraud is the same thing as isfraud so we drop


In [ ]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [ ]:
df.shape #the record number with 11 column


(6362620, 11)

going to drop isflaggedfraud beacuse it the same thing as fraud. and nameDest since we dont need the id of the transaction

In [ ]:
df1 = df.drop(['isFlaggedFraud','nameOrig'], axis=1)

(6362620, 11)

In [ ]:
df1['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [ ]:
df1

,step,type,amount,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.00,160296.36,M1979787155,0.00,0.00,0
1,1,PAYMENT,1864.28,21249.00,19384.72,M2044282225,0.00,0.00,0
2,1,TRANSFER,181.00,181.00,0.00,C553264065,0.00,0.00,1
3,1,CASH_OUT,181.00,181.00,0.00,C38997010,21182.00,0.00,1
4,1,PAYMENT,11668.14,41554.00,29885.86,M1230701703,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,339682.13,0.00,C776919290,0.00,339682.13,1
6362616,743,TRANSFER,6311409.28,6311409.28,0.00,C1881841831,0.00,0.00,1
6362617,743,CASH_OUT,6311409.28,6311409.28,0.00,C1365125890,68488.84,6379898.11,1
6362618,743,TRANSFER,850002.52,850002.52,0.00,C2080388513,0.00,0.00,1


Handling missing values  

In [ ]:
#checking the number of missing values
df1.isnull().sum()

step              0
type              0
amount            0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
dtype: int64

In [ ]:
# distribution of legit transactions & fraudulent transactions
df1['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

checking if there are duplicates

In [ ]:
duplicates = df.duplicated()
print(duplicates)

0          False
1          False
2          False
3          False
4          False
           ...  
6362615    False
6362616    False
6362617    False
6362618    False
6362619    False
Length: 6362620, dtype: bool


In [ ]:
total_duplicates = df1.duplicated().sum()
print(f'Total duplicates in the DataFrame: {total_duplicates}')

Total duplicates in the DataFrame: 0


Save cleaned data into here

In [ ]:
df1.to_csv('cleaned_data2.0.csv', index=False)
from google.colab import files
files.download('cleaned_data2.0.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>